In [ ]:
import os
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'

import seaborn
seaborn.set_style('darkgrid')
matplotlib.rcParams['figure.figsize'] = (16.0, 1.0)
matplotlib.rcParams['font.size'] = 7


In [ ]:
#! wget --password nedc_tuh_eeg --user nedc_tuh_eeg https://www.isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg/v0.6.0/edf/001/00000010.tar.gz

In [ ]:
#! tar -xvf 00000010.tar.gz

In [ ]:
import mne.io

In [ ]:
filename = 'tuh_eeg/v0.6.0/edf/001/00000010/s02_2012_03_01/a_3.edf'
dataset =  mne.io.read_raw_edf(filename)

In [ ]:
dataset.load_data()

In [ ]:
signal = dataset.get_data()

Find out sampling rate by looking at timestamps

In [ ]:
import numpy as np
np.unique(np.int32(1.0 / np.diff(dataset.times))) # -> sampling rate 250

#### Windowed FFT

In [ ]:
def extract_windows(signal, win_len, win_stride, axis):
    signal_crops = []
    for i_start in range(0, signal.shape[axis] - win_len+1, win_stride):
        signal_crops.append(np.take(signal, range(i_start,i_start+win_len), axis=axis,))
    return np.array(signal_crops)

In [ ]:
windows = extract_windows(signal, win_len=500,win_stride=500,axis=1)

ffted =  np.fft.rfft(windows, axis=-1)

amplitudes = np.abs(ffted)

### Plot median

In [ ]:
median_amp = np.median(amplitudes, axis=0)

In [ ]:
freqs = np.fft.rfftfreq(n=500, d=1/250.0)
i_chan = dataset.ch_names.index('EEG O1-REF')
plt.plot(freqs, median_amp[i_chan])
plt.figure()
plt.plot(freqs, median_amp[i_chan])
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')

In [ ]:
freqs = np.fft.rfftfreq(n=500, d=1/250.0)
i_chan = dataset.ch_names.index('EEG O2-REF')
plt.plot(freqs, median_amp[i_chan])
plt.figure()
plt.plot(freqs, median_amp[i_chan])
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')

### Plot mean

In [ ]:
mean_amp = np.mean(amplitudes, axis=0)

In [ ]:
freqs = np.fft.rfftfreq(n=500, d=1/250.0)
i_chan = dataset.ch_names.index('EEG O1-REF')
plt.plot(freqs, mean_amp[i_chan])
plt.figure()
plt.plot(freqs, mean_amp[i_chan])
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')

In [ ]:
plt.plot(signal[i_chan]);

In [ ]:
plt.plot(signal[i_chan][0:2000])

In [ ]:
plt.plot(signal[i_chan][-2000:]);

### Plot mean fft without last part

In [ ]:
windows = extract_windows(signal[:,:-500], win_len=500,win_stride=500,axis=1)
ffted =  np.fft.rfft(windows, axis=-1)
amplitudes = np.abs(ffted)
mean_amp = np.mean(amplitudes, axis=0)

freqs = np.fft.rfftfreq(n=500, d=1/250.0)
i_chan = dataset.ch_names.index('EEG O1-REF')
plt.plot(freqs, mean_amp[i_chan])
plt.figure()
plt.plot(freqs, mean_amp[i_chan])
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')